In [1]:
import os

import numpy as np
import pandas as pd

# import warnings
# warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

%matplotlib inline
import sklearn
import tensorflow as tf
import tensorflow_addons as tfa
from PIL import Image
from tensorflow import keras
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.layers import (
    AveragePooling2D,
    AvgPool2D,
    Conv2D,
    Dense,
    Flatten,
    GlobalAveragePooling2D,
    MaxPooling2D,
)
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# from venn import venn

In [2]:
PATH = "/app/_data/"
BATCH_SIZE = 64
SEED = 42
IMAGE_SIZE = (224, 224)
NUM_CLASSES = 6
INPUT_SIZE = (224, 224, 3)

In [3]:
labels = pd.read_csv(PATH + "train.csv")
sample_submission = pd.read_csv(PATH + "sample_submission.csv")

In [4]:
sample_submission

,image,labels
0,85f8cb619c66b863.jpg,healthy
1,ad8770db05586b59.jpg,healthy
2,c7b03e718489f3ca.jpg,healthy


In [5]:
labels.head()
labels.info()

,image,labels
0,800113bb65efe69e.jpg,healthy
1,8002cb321f8bfcdf.jpg,scab frog_eye_leaf_spot complex
2,80070f7fb5e2ccaa.jpg,scab
3,80077517781fb94f.jpg,scab
4,800cbf0ff87721f8.jpg,complex


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18632 entries, 0 to 18631
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image   18632 non-null  object
 1   labels  18632 non-null  object
dtypes: object(2)
memory usage: 291.2+ KB


In [6]:
labels["labels"].value_counts(normalize=True)

scab                               0.259017
healthy                            0.248175
frog_eye_leaf_spot                 0.170728
rust                               0.099828
complex                            0.085981
powdery_mildew                     0.063547
scab frog_eye_leaf_spot            0.036818
scab frog_eye_leaf_spot complex    0.010734
frog_eye_leaf_spot complex         0.008856
rust frog_eye_leaf_spot            0.006441
rust complex                       0.005206
powdery_mildew complex             0.004669
Name: labels, dtype: float64

In [7]:
labels = labels.join(labels["labels"].str.get_dummies(sep=" "))

In [8]:
feature_columns = labels.columns[2:].tolist()
feature_columns

['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']

## resizing and saving images

In [9]:
# def resize_im(
#     image_name, new_folder_name="small_nearest", size=(224, 224), resample=Image.NEAREST
# ):
#     image = Image.open(PATH + "train_images/" + image_name)
#     image = image.resize(size, resample=resample)
#     if not os.path.isdir(PATH + "train_images/" + new_folder_name + "/"):
#         os.mkdir(PATH + "train_images/" + new_folder_name + "/")
#     image.save(PATH + "train_images/" + new_folder_name + "/" + image_name)
# labels['image'].apply(resize_im)

## weights

In [10]:
# labels["multi_label"] = labels["labels"].astype("category").cat.codes
# dict_weights = (
#     1
#     / labels["multi_label"].value_counts()
#     / np.sum(1 / labels["multi_label"].value_counts())
# ).to_dict()
# def weight_fill(x):
#    return dict_weights[x]
# labels['weights'] = labels["multi_label"].apply(weight_fill)

## ImageDataGenerator

In [11]:
train_datagen = ImageDataGenerator(
    validation_split=0.2,
    rescale=1.0 / 255,
    # #     samplewise_center=True,
    # #     samplewise_std_normalization=True,
#         rotation_range = 45,width_shift_range=0.25,
#         height_shift_range=0.25,
#         brightness_range=(0.5, 1.5),
#         shear_range = 45,
#         zoom_range = 0.25,
#         horizontal_flip=True,
#         vertical_flip=True
)
train = train_datagen.flow_from_dataframe(
    dataframe=labels,
    directory=PATH + "train_images/small_nearest/",
    x_col="image",
    y_col=feature_columns,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="raw",
    subset="training",
    seed=SEED,
    interpolation = 'nearest'
)

Found 14906 validated image filenames.


In [12]:
valid_datagen = ImageDataGenerator(
    validation_split=0.2,
    rescale=1.0 / 255,
)
valid = valid_datagen.flow_from_dataframe(
    dataframe=labels,
    directory=PATH + "train_images/small_nearest/",
    x_col="image",
    y_col=feature_columns,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="raw",
    subset="validation",
    seed=SEED,
    interpolation = 'nearest'
)

Found 3726 validated image filenames.


In [13]:
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test = test_datagen.flow_from_dataframe(
    dataframe=sample_submission,
    directory=PATH + "test_images/",
    x_col="image",
    y_col=None,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode=None,
    seed=SEED,
    shuffle=False,
    interpolation = 'nearest'
)

Found 3 validated image filenames.


### plots

In [14]:
def plot_curves(model, loss=False):
    df = pd.DataFrame(model.history)
    if loss:
        plt.figure(figsize=(12, 6))
        plt.title("loss")
        plt.plot(df.iloc[:, 0], label="loss")
        plt.plot(df.iloc[:, 5], label="val_loss")
        plt.legend()
        plt.show()
    else:
        plt.figure(figsize=(12, 20))
        plt.subplot(4, 1, 1)
        plt.title("accuracy")
        plt.plot(df.iloc[:, 1], label="accuracy")
        plt.plot(df.iloc[:, 6], label="val_accuracy")
        plt.legend()

        plt.subplot(4, 1, 2)
        plt.title("recall")
        plt.plot(df.iloc[:, 2], label="recall")
        plt.plot(df.iloc[:, 7], label="val_recall")
        plt.legend()

        plt.subplot(4, 1, 3)
        plt.title("precision")
        plt.plot(df.iloc[:, 3], label="precision")
        plt.plot(df.iloc[:, 8], label="val_precision")
        plt.legend()

        plt.subplot(4, 1, 4)
        plt.title("f1")
        plt.plot(df.iloc[:, 4], label="f1")
        plt.plot(df.iloc[:, 9], label="val_f1")
        plt.legend()
        plt.show();

## model

### base_model

In [15]:
# backbone = ResNet50(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
# model_w = Sequential()
# model_w.add(backbone)
# model_w.add(GlobalAveragePooling2D())
# model_w.add(Dense(units=7, activation="sigmoid"))
# model_w.compile(
#     loss="CategoricalCrossentropy",
#     optimizer=Adam(lr=0.001),
#     metrics=[keras.metrics.CategoricalAccuracy(), keras.metrics.Recall(), keras.metrics.Precision(), tfa.metrics.F1Score(
#                     num_classes=7,
#                     average='macro')]
# )

### from keras
<https://keras.io/guides/preprocessing_layers/#image-preprocessing-layers>

In [13]:
data_augmentation = keras.Sequential(
    [
        keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        keras.layers.experimental.preprocessing.RandomHeight(0.1),
        keras.layers.experimental.preprocessing.RandomWidth(0.1),
        keras.layers.experimental.preprocessing.RandomRotation(0.15),
        keras.layers.experimental.preprocessing.RandomZoom(0.25, 0.25),
    ]
)
#         keras.preprocessing.image.random_shear(45),
#         keras.preprocessing.image.random_rotation(45),
#         keras.preprocessing.image.random_brightness(0.5, 1.5),
#         keras.preprocessing.image.random_zoom(0.75, 1.25),
#         keras.preprocessing.image.random_shift(0.25, 0.25)
# x = keras.preprocessing.image.random_shear(x, 45, row_axis=0, col_axis=1, channel_axis=2),
# x = keras.preprocessing.image.random_rotation(x, 45, row_axis=0, col_axis=1, channel_axis=2),
# x = keras.preprocessing.image.random_brightness(x, (0.5, 1.5)),
# x = keras.preprocessing.image.random_zoom(x, (0.75, 1.25), row_axis=0, col_axis=1, channel_axis=2),
# x = keras.preprocessing.image.random_shift(x, 0.25, 0.25, row_axis=0, col_axis=1, channel_axis=2)

In [17]:
def preprocess_data(x, y):
    x_p = data_augmentation(x)
    x_p = keras.applications.resnet.preprocess_input(x_p)
    return (x_p, y)

In [18]:
# preprocess_data(next(valid)[0])

### func style

In [14]:
inputs = keras.Input(shape=INPUT_SIZE)
x = data_augmentation(inputs)
x = keras.applications.resnet.preprocess_input(x)
x = keras.applications.ResNet50(weights="imagenet", include_top=False)(x)
x = keras.layers.GlobalAveragePooling2D(name="avg_pool")(x)
x = keras.layers.Flatten(name="flatten")(x)
outputs = keras.layers.Dense(NUM_CLASSES, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [15]:
model.compile(
    loss="binary_crossentropy",
    optimizer=Adam(lr=0.0005),
    metrics=['acc',
        keras.metrics.Recall(),
        keras.metrics.Precision(),
        tfa.metrics.F1Score(num_classes=NUM_CLASSES, average="micro"),
    ],
)

In [16]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, None, None, 3)     0         
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, None, None, 3)]   0         
_________________________________________________________________
tf_op_layer_BiasAdd (TensorF [(None, None, None, 3)]   0         
_________________________________________________________________
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 2048)              0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)             

In [17]:
history = model.fit(
    train,
    validation_data=valid,
    epochs=50,
    batch_size=BATCH_SIZE,
    verbose=1,
    use_multiprocessing=True,
)

Epoch 1/50
233/233 [==============================] - 115s 494ms/step - loss: 0.2610 - acc: 0.6844 - recall: 0.6054 - precision: 0.7407 - f1_score: 0.6805 - val_loss: 0.8476 - val_acc: 0.1071 - val_recall: 0.1098 - val_precision: 0.1167 - val_f1_score: 0.1132
Epoch 2/50
233/233 [==============================] - 84s 360ms/step - loss: 0.1848 - acc: 0.7931 - recall: 0.7417 - precision: 0.8296 - f1_score: 0.7841 - val_loss: 0.5085 - val_acc: 0.2185 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_f1_score: 0.2118
Epoch 3/50
233/233 [==============================] - 83s 355ms/step - loss: 0.1634 - acc: 0.8230 - recall: 0.7720 - precision: 0.8515 - f1_score: 0.8129 - val_loss: 0.6521 - val_acc: 0.1071 - val_recall: 0.1098 - val_precision: 0.1167 - val_f1_score: 0.1132
Epoch 4/50
233/233 [==============================] - 80s 342ms/step - loss: 0.1530 - acc: 0.8339 - recall: 0.7918 - precision: 0.8574 - f1_score: 0.8237 - val_loss: 2.5901 - val_acc: 0.1071 - val_recall: 0.1747 - 

Process Keras_worker_ForkPoolWorker-132:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 131, in worker
    put((job, i, result))
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 362, in put
    obj = _ForkingPickler.dumps(obj)
  File "/usr/lib/python3.8/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
plot_curves(history)

In [ ]:
plot_curves(history, loss=True)

## test

In [ ]:
prediction = model.predict(test) > 0.5

In [ ]:
test_sub = pd.DataFrame(prediction, columns=feature_columns)
for name in test_sub.columns.tolist():
    test_sub[name] = test_sub[name].replace({True: name, False: ""})
test_sub["all"] = test_sub.apply(" ".join, axis=1).str.split().str.join(sep=" ")
test_sub.to_csv("app/sandbox/test_sub.csv")

In [ ]:
backbone = ResNet50(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
model = Sequential()
model.add(backbone)
model.add(GlobalAveragePooling2D())
model.add(Dense(units=6, activation="sigmoid"))
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(lr=0.0001),
    metrics=[
        keras.metrics.CategoricalAccuracy(),
        keras.metrics.Recall(),
        keras.metrics.Precision(),
        tfa.metrics.F1Score(num_classes=6, average="micro"),
    ],
)

In [ ]:
history = model.fit(
    train,
    validation_data=valid,
    epochs=3,
    batch_size=BATCH_SIZE,
    #     steps_per_epoch=64,
    #     validation_steps=16,
    verbose=1,
    use_multiprocessing=True,
)

In [ ]:
model_l = keras.models.load_model('/app/_data/models/base_model')

In [ ]:
a = model_l.predict(valid)

In [ ]:
a[0]